In [1]:
%cd ../

%load_ext autoreload
%autoreload 2

/home/hoanghu/projects/Food-Waste-Optimization


In [2]:
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

# Load data

In [3]:
path_lunches = Path("src/data/basic_mvp_data/Sold lunches.csv")

raw_lunch = pd.read_csv(path_lunches, delimiter=';', encoding='utf-8-sig', parse_dates=['Date'], low_memory=False)

In [4]:
path = "src/data/basic_mvp_data/Biowaste.csv"

biowaste = pd.read_csv(path, delimiter=';')

In [5]:
path = "src/data/basic_mvp_data/tuntidata2.xlsx"

receipts = pd.read_excel(path, header=0)

receipts.head()

,Date,Kuitin tunti,Ravintola,Kuitti kpl
0,2023-01-02,8,600 Chemicum,1
1,2023-01-02,10,600 Chemicum,18
2,2023-01-02,11,600 Chemicum,83
3,2023-01-02,12,600 Chemicum,79
4,2023-01-02,13,600 Chemicum,90


In [6]:
path = "src/data/basic_mvp_data/supersight.xlsx"

occupancy = pd.read_excel(path, header=0, index_col=None)

occupancy.head()

,dateCreated,countIn,countOut,phoneName
0,2024-03-15T04:26:15.000Z,0,1,S63
1,2024-03-15T04:26:24.000Z,1,2,S63
2,2024-03-15T04:26:27.000Z,1,0,S63
3,2024-03-15T05:19:19.000Z,1,0,S63
4,2024-03-15T05:37:09.000Z,1,0,S63


In [7]:
path = "src/data/basic_mvp_data/kumpula_lounaat_kat.csv"

meals = pd.read_csv(path, delimiter=';', header=None)

meals.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,NaN,Ravintola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,600 Chemicum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total
2,NaN,Kala,%-osuus,Kana,%-osuus,Kasvis,%-osuus,Liha,%-osuus,Not Mapped,...,Kasvis,%-osuus,Liha,%-osuus,Not Mapped,%-osuus,Vegaani,%-osuus,Total,NaN
3,NaN,Lounaat ilman lisämyyntiä,NaN,Lounaat ilman lisämyyntiä,NaN,Lounaat ilman lisämyyntiä,NaN,Lounaat ilman lisämyyntiä,NaN,Lounaat ilman lisämyyntiä,...,Lounaat ilman lisämyyntiä,NaN,Lounaat ilman lisämyyntiä,NaN,Lounaat ilman lisämyyntiä,NaN,Lounaat ilman lisämyyntiä,NaN,Lounaat ilman lisämyyntiä,Lounaat ilman lisämyyntiä
4,Total,73 554,"26,4 %",47 233,"16,9 %",1 524,"0,5 %",41 839,"15,0 %",14 822,...,3 910,"5,3 %",7 744,"10,6 %",195,"0,3 %",31 626,"43,2 %",73 132,397 770


# 2. Pre-process

## With `biowaste`

In [8]:
biowaste['Date'] = biowaste['Date'].apply(lambda x: datetime.strptime(x, r"%d.%m.%Y"))

biowaste.loc[:, 'Ravintola'] = biowaste['Ravintola'].replace({
    "600 Chemicum": "Chemicum",
    "610 Physicum": "Physicum",
    "620 Exactum": "Exactum"
})

biowaste = biowaste.rename(columns={
    'Date': 'date',
    'Ravintola': 'restaurant', 
    'Asiakasbiojäte. tiski (kg)': 'amnt_waste_customer', 
    'Biojäte kahvi. porot (kg)': 'amnt_waste_coffee', 
    'Keittiön biojäte (ruoanvalmistus) (kg)': 'amnt_waste_kitchen',
    'Salin biojäte (jämät) (kg)': 'amnt_waste_hall',
})

# Filter out rows which all 4 types of waste have zero amount
cols = ['amnt_waste_customer', 'amnt_waste_coffee', 'amnt_waste_kitchen', 'amnt_waste_hall']
tmp = biowaste[cols].sum(axis=1)
biowaste = biowaste[tmp > 0]

biowaste.head()

,date,restaurant,amnt_waste_customer,amnt_waste_coffee,amnt_waste_kitchen,amnt_waste_hall
3,2023-01-02,Chemicum,4.70,1.2,12.0,0.0
6,2023-01-03,Chemicum,5.00,1.4,14.8,0.0
9,2023-01-04,Chemicum,4.15,4.0,7.1,0.0
12,2023-01-05,Chemicum,10.00,3.3,8.5,0.0
24,2023-01-09,Chemicum,7.65,2.1,4.9,0.0


## With `receipts`

In [9]:
receipts.loc[:, 'Ravintola'] = receipts['Ravintola'].replace({
    "600 Chemicum": "Chemicum",
    "610 Physicum": "Physicum",
    "620 Exactum": "Exactum"
})
    
receipts = receipts.rename(columns={
    'Date': 'date',
    'Kuitin tunti': 'hour',
    'Ravintola': 'restaurant',
    'Kuitti kpl': 'num_rcpts'
})

receipts = receipts.groupby(['date', 'restaurant'])['num_rcpts'].sum().reset_index()

receipts.head()

,date,restaurant,num_rcpts
0,2023-01-02,Chemicum,272
1,2023-01-03,Chemicum,327
2,2023-01-04,Chemicum,351
3,2023-01-04,Physicum,1
4,2023-01-05,Chemicum,437


## With `occupancy`

In [10]:
phoneName2restaurant = {
    'S163': 'Exactum',
    'S216': 'Chemicum',
    'S217': 'Physicum',
}

occupancy = occupancy[occupancy['phoneName'].isin(phoneName2restaurant.keys())]
occupancy['restaurant'] = occupancy['phoneName'].apply(lambda x: phoneName2restaurant[x])

# Convert index
occupancy['date'] = pd.to_datetime(occupancy['dateCreated']).dt.tz_localize(None)
occupancy = occupancy.set_index('date')

# Accumulate countIn, countOut by day
cols = ['countIn', 'countOut', 'restaurant']
occupancy = occupancy[cols].groupby(by='restaurant').resample('D').sum()

# Reset date
occupancy = occupancy[['countIn', 'countOut']].reset_index()

# Rename columns' name
occupancy = occupancy.rename(columns={
    'countIn': 'num_customer_in',
    'countOut': 'num_customer_out'
})

occupancy.head()

,restaurant,date,num_customer_in,num_customer_out
0,Chemicum,2024-05-27,11,6
1,Chemicum,2024-05-28,767,709
2,Chemicum,2024-05-29,671,677
3,Chemicum,2024-05-30,717,697
4,Chemicum,2024-05-31,689,668


## With `meals`

In [11]:
cols_name = [
    'num_fish', 'percent_fish', 
    'num_chicken', 'percent_chicken',
    'num_vegetarian', 'percent_vegetarian', 
    'num_meat', 'percent_meat',
    'num_NotMapped', 'percent_NotMapped',
    'num_vegan', 'percent_vegan',
    'total' 
]
len_cols_per_restaurant = len(cols_name) - 1

restaurants = ['Chemicum', 'Physicum', 'Exactum']

cols_important = ['date', 'restaurant', 'num_fish', 'num_chicken', 'num_vegetarian', 'num_meat', 'num_NotMapped', 'num_vegan']

def _f_process(x: str):
    x = x.replace(' %', '')
    x = x.replace(' ', '')
    x = x.replace(',', '.')
    
    ret = float(x)

    return ret

In [12]:
# Get date column
dates = meals.loc[5:, 0].apply(lambda x: datetime.strptime(x, r"%Y-%m-%d"))

# Extract restaurant sold meals data
data_restaurants = []

idx_col = 1
for restaurant in restaurants:
    df_restaurant = meals.loc[5:, idx_col:idx_col + len_cols_per_restaurant].copy()

    df_restaurant = df_restaurant.set_axis(cols_name, axis=1)

    df_restaurant = df_restaurant.map(_f_process)

    df_restaurant['restaurant'] = restaurant
    df_restaurant['date'] = dates.copy()

    df_restaurant = df_restaurant[cols_important]
    
    data_restaurants.append(df_restaurant)

    idx_col += len_cols_per_restaurant

meals = pd.concat(data_restaurants)

meals.head()


,date,restaurant,num_fish,num_chicken,num_vegetarian,num_meat,num_NotMapped,num_vegan
5,2023-01-02,Chemicum,85.0,0.0,0.0,171.0,1.0,91.0
6,2023-01-03,Chemicum,163.0,0.0,32.0,78.0,1.0,120.0
7,2023-01-04,Chemicum,70.0,0.0,0.0,218.0,3.0,137.0
8,2023-01-05,Chemicum,232.0,85.0,0.0,2.0,4.0,178.0
9,2023-01-09,Chemicum,107.0,0.0,0.0,264.0,0.0,207.0


## With 'lunches'

In [13]:
lunches = (
    raw_lunch
    .replace(
        {
            '600 Chemicum': 'Chemicum',
            '610 Physicum': 'Physicum',
            '620 Exactum': 'Exactum',
            'Kala': 'fish',
            'Liha': 'meat',
            'Vegaani': 'vegan',
            'Kasvis': 'vegetarian',
            'Kana': 'chicken'
        },
    )
    .rename(columns={
        'Date': 'date',
        'Restaurant': 'restaurant',
        'Food Category': 'category',
        'Dish': 'meal'
    })
    
)

lunches['pcs'] = pd.to_numeric(lunches.pcs, errors='coerce')
lunches['date'] = pd.to_datetime(lunches.date, format='%d.%m.%Y', errors='coerce')

lunches = (
    lunches
    .groupby(['date', 'restaurant', 'category', 'meal'])['pcs']
    .sum()
    .reset_index()
)

# Remove 'Not Napped'
lunches = lunches[lunches['category'] != 'Not Mapped']

# Remove 'takeaway'
def is_takeaway(s: str):
    return s.lower().count('take away') > 0

lunches['is_takeaway'] = lunches['meal'].map(is_takeaway)
lunches = lunches[~lunches['is_takeaway']].drop(columns='is_takeaway')

# Add dish name processing
def _f(s: str):
    s = s.split(',')[0]
    s = s.split('&')[0]
    s = s.strip()

    return s

lunches['meal'] = lunches['meal'].apply(_f)

# Sum pieces again since on some dates, same dish appears twice
lunches = lunches.groupby(['date', 'restaurant', 'category', 'meal'])['pcs'].sum().reset_index()


lunches.head(10)

,date,restaurant,category,meal,pcs
0,2023-01-02,Chemicum,fish,Kalapuikot tillikermaviilikast,78.0
1,2023-01-02,Chemicum,meat,Uunimakkaraa,165.0
2,2023-01-02,Chemicum,vegan,Marokkolainen linssipata,84.0
3,2023-01-03,Chemicum,fish,Herkkulohipihvit,105.0
4,2023-01-03,Chemicum,fish,Kalapuikot tillikermaviilikast,52.0
5,2023-01-03,Chemicum,meat,Pasta Carbonara,17.0
6,2023-01-03,Chemicum,meat,Uunimakkaraa,56.0
7,2023-01-03,Chemicum,vegan,Marokkolainen linssipata,62.0
8,2023-01-03,Chemicum,vegan,Vegaaninen buttertofu,51.0
9,2023-01-03,Chemicum,vegetarian,Feta-pinaattilasagnette,29.0


# Process

### Create table containing dishes' name and meal quantity by date and restaurant

In [14]:
lunches = (
    lunches
    .groupby(['date', 'restaurant', 'category'])
    .agg({
        'meal': lambda x: [x] if isinstance(x, str) else x,
    })
    .reset_index()
)

lunches['category'] = lunches['category'].map(lambda x: f"meals_{x}")
lunches['meal'] = lunches['meal'].map(lambda x: np.array([x]) if isinstance(x, str) else x)

lunches = lunches.pivot(index=['date', 'restaurant'], columns='category', values='meal')

lunches.head()

,category,meals_chicken,meals_fish,meals_meat,meals_vegan,meals_vegetarian
date,restaurant,,,,,
2023-01-02,Chemicum,NaN,[Kalapuikot tillikermaviilikast],[Uunimakkaraa],[Marokkolainen linssipata],NaN
2023-01-03,Chemicum,NaN,"[Herkkulohipihvit, Kalapuikot tillikermaviilik...","[Pasta Carbonara, Uunimakkaraa]","[Marokkolainen linssipata, Vegaaninen buttertofu]",[Feta-pinaattilasagnette]
2023-01-04,Chemicum,NaN,"[Herkkulohipihvit, Rapea kalaleike]",[Lihapullat],[Punajuuripyörykät],NaN
2023-01-05,Chemicum,[Kievin kana],"[Rapea kalaleike, Sitruunaiset kalapalat]",NaN,[Meksikon Beanit Chilipata],NaN
2023-01-09,Chemicum,NaN,"[Kalapuikot tillikermaviilikast, Sitruunaiset ...",[Chorizo lihap ja ruskkastike],"[Kasvis-jalapnuget ja tomatsals, Kasvisjauhisp...",NaN


## With 'fact'

In [22]:
date_start = pd.to_datetime('2023-01-02')
date_end = pd.to_datetime('2024-07-02')

date_range = pd.date_range(date_start, date_end, freq='B')

fact = pd.concat(
    [
        pd.DataFrame({
            'date': date_range,
            'restaurant': restaurant
        })
        for restaurant in restaurants
    ]
)

foreign_keys = ['restaurant', 'date']

fact = (
    fact
    .merge(meals, on=foreign_keys, how='inner')
    .merge(occupancy, on=foreign_keys, how='left')
    .merge(receipts, on=foreign_keys, how='inner')
    .merge(biowaste, on=foreign_keys, how='inner')
    .merge(lunches, on=foreign_keys, how='inner')
)

fact.head()

,date,restaurant,num_fish,num_chicken,num_vegetarian,num_meat,num_NotMapped,num_vegan,num_customer_in,num_customer_out,num_rcpts,amnt_waste_customer,amnt_waste_coffee,amnt_waste_kitchen,amnt_waste_hall,meals_chicken,meals_fish,meals_meat,meals_vegan,meals_vegetarian
0,2023-01-02,Chemicum,85.0,0.0,0.0,171.0,1.0,91.0,NaN,NaN,272,4.70,1.2,12.0,0.0,NaN,[Kalapuikot tillikermaviilikast],[Uunimakkaraa],[Marokkolainen linssipata],NaN
1,2023-01-03,Chemicum,163.0,0.0,32.0,78.0,1.0,120.0,NaN,NaN,327,5.00,1.4,14.8,0.0,NaN,"[Herkkulohipihvit, Kalapuikot tillikermaviilik...","[Pasta Carbonara, Uunimakkaraa]","[Marokkolainen linssipata, Vegaaninen buttertofu]",[Feta-pinaattilasagnette]
2,2023-01-04,Chemicum,70.0,0.0,0.0,218.0,3.0,137.0,NaN,NaN,351,4.15,4.0,7.1,0.0,NaN,"[Herkkulohipihvit, Rapea kalaleike]",[Lihapullat],[Punajuuripyörykät],NaN
3,2023-01-05,Chemicum,232.0,85.0,0.0,2.0,4.0,178.0,NaN,NaN,437,10.00,3.3,8.5,0.0,[Kievin kana],"[Rapea kalaleike, Sitruunaiset kalapalat]",NaN,[Meksikon Beanit Chilipata],NaN
4,2023-01-09,Chemicum,107.0,0.0,0.0,264.0,0.0,207.0,NaN,NaN,529,7.65,2.1,4.9,0.0,NaN,"[Kalapuikot tillikermaviilikast, Sitruunaiset ...",[Chorizo lihap ja ruskkastike],"[Kasvis-jalapnuget ja tomatsals, Kasvisjauhisp...",NaN


In [10]:
# fact.to_excel("fact.xlsx", index=False)